In [49]:
import networkx as nx
import numpy as np


class SpatialNetwork():
    def __init__(self, n, k, graph_type='regular', dep = None, snowdrift=False):
        """creates spatial network"""
        self.n = n
        self.k = k
        self.dep = dep
        self.G = self.create_graph(graph_type)

    def create_graph(self, graph_type):
        if graph_type == 'lattice':
            return self.create_lattice()
        elif graph_type == 'powerlaw':
            return nx.powerlaw_cluster_graph(self.n, self.k, 0)
        elif graph_type == 'HK':
            if self.dep != None:
                return nx.powerlaw_cluster_graph(self.n, self.k, self.dep)
            else:
                return nx.powerlaw_cluster_graph(self.n, self.k, .3)
        G = self.create_regular()
        if graph_type == 'regular':
            return G
        elif graph_type == 'ER':
            return self.rewire(G, p=.3)
        elif graph_type == 'random':
            return self.rewire(G, p=1)
        else:
            raise ValueError('not a recognized graph type')

    def create_regular(self):
        """
        creates regular graph. Is a helper function for rewired graphs. Some
        code taken from/based on jupyter notebook chap3, credit to Allen Downey
        """
        G = nx.Graph()
        quo, rem = divmod(self.k, 2)
        nodes = list(range(self.n))
        G.add_nodes_from(nodes)
        G.add_edges_from(adjacent_edges(nodes, quo))
        # if k is odd, add opposite edges
        if rem:
            if self.n%2:
                msg = "Can't make a regular graph if n and k are odd."
                raise ValueError(msg)
            G.add_edges_from(opposite_edges())
        return G

    def create_lattice(self):
        """
        creates a lattice graph with n nodes that is k long
        """
        G = nx.Graph()
        nodes = list(range(self.n))
        G.add_nodes_from(nodes)
        for node in nodes:
            if node + self.k in nodes:
                G.add_edge(node, node + self.k)
            if node % self.k != (self.k-1) and node + 1 in nodes:
                G.add_edge(node, node + 1)
        return G

    def rewire(self, G, p):
        """Rewires each edge with probability `p`.
        code taken from/based on jupyter notebook chap3, credit to Allen Downey
        G: Graph
        p: float
        """
        if self.dep != None:
            p = self.dep
        nodes = set(G.nodes())
        for edge in G.edges():
            if flip(p):
                u, v = edge
                choices = nodes - {u} - set(G[u])
                new_v = np.random.choice(tuple(choices))
                G.remove_edge(u, v)
                G.add_edge(u, new_v)
        return G


def adjacent_edges(nodes, halfk):
    """
    code taken from/based on jupyter notebook chap3, credit to Allen Downey
    """
    n = len(nodes)
    for i, u in enumerate(nodes):
        for j in range(i+1, i+halfk+1):
            v = nodes[j % n]
            yield u, v


def opposite_edges(nodes):
    """Enumerates edges that connect opposite nodes.
    code taken from/based on jupyter notebook chap3, credit to Allen Downey
    """
    n = len(nodes)
    for i, u in enumerate(nodes):
        j = i + n//2
        v = nodes[j % n]
        yield u, v


def flip(p):
    """Returns True with probability `p`."""
    return np.random.random() < p

In [54]:
class Node:
    C, D = 'C', 'D'
    T, R, P, S = 3, 1, 0, 0

    def __init__(self, state):
        """
        node object in graph. state is C or D
        """
        self.state = state
        self.score = 0  # will hold value calculated based on prisoners dilemma

    @classmethod
    def is_cooperator_state(cls, state):
        return state == cls.C

    def is_cooperator(self):
        return self.is_cooperator_state(self.state)

    def add_score(self, score):
        self.score += score

    def reset_score(self):
        self.score = 0

    # Play a single PD with neighbors, and calculate score
    #  neighbors: The list of Node object
    def play_pd(self, neighbors):
        """
        timestep for node
        """
        self.reset_score()
        for neighbor in neighbors:
            self.add_score(self.pd_score(neighbor.state))

    # play single PD
    def pd_score(self, neighbor_state):
        if self.is_cooperator():
            if self.is_cooperator_state(neighbor_state):
                return Node.R
            return Node.S
        else:
            if self.is_cooperator_state(neighbor_state):
                return Node.T
            return Node.P

    # Copy the state of the most successful neighbor
    def get_max_state(self, neighbors):
        max_score = max([neighbor.score for neighbor in neighbors])
        self.max_state = [neighbor.state for neighbor in neighbors if neighbor.score == max_score]
    
    def update_state(self):
        self.state = self.max_state[0]

In [56]:
import random

# It would be better to construct a class for

def proceed_one_stage():
    # play a single PD
    for i in range(n):
        neighbor = [nodes[key] for key in Graph[i]]
        nodes[i].play_pd(neighbor)
        
    # update state
    for i in range(n):
        neighbor = [nodes[key] for key in Graph[i]]
        nodes[i].get_max_state(neighbor)
        
    for i in range(n):
        nodes[i].update_state()

    c_ratio = len([node for node in nodes if node.is_cooperator()]) / n
    return c_ratio

# TODO: create rules
n = 3600  # the number of node
k = 8

# TODO: create node object
p = 0.98  # the initial ratio of cooperator
# initial state distribution(C/D) should be applied.

N = 50  # the number of steps
ave = 0
c_ratio = 0
for _ in range(10):
    cooperators = random.sample(range(n), int(n * p))
    nodes = [Node('C') if i in cooperators else Node('D') for i in range(n)]

    G = SpatialNetwork(n, k, graph_type='ER')
    Graph = G.G

# TODO: create test suite
    for _ in range(N):
        c_ratio = proceed_one_stage()
    print("c ratio: {}".format(c_ratio))
    ave = ave + c_ratio
ave/10

c ratio: 0.0019444444444444444
c ratio: 0.0002777777777777778
c ratio: 0.0002777777777777778
c ratio: 0.0016666666666666668
c ratio: 0.0002777777777777778
c ratio: 0.0030555555555555557
c ratio: 0.0008333333333333334
c ratio: 0.001388888888888889
c ratio: 0.0
c ratio: 0.0030555555555555557


0.0012777777777777779